# Scope 8 – Hansen–Jagannathan bound as in-sample Sharpe cap

We use the **in-sample Hansen–Jagannathan (HJ) bound** (maximum Sharpe ratio in the asset space) as an upper limit on the Sharpe ratio we allow when selecting in-sample portfolios. Portfolios that would otherwise have an estimated Sharpe above this bound are capped at the bound (via a mix of GMV and tangency on the efficient frontier). We then compare **unconstrained tangency** vs **HJ-capped** portfolios out-of-sample: realised Sharpe and whether they remain on the OOS efficient frontier.

## How the HJ bound was calculated

We use the **maximum-Sharpe formula** in the asset space:

- **Excess mean**: \( \mu_e = \mu - r_f \mathbf{1} \) (for 30 industries we use sample μ and average in-sample risk-free rate; for FF5 factors are already excess returns so \( r_f = 0 \), \( \mu_e = \mu \)).
- **HJ bound (max Sharpe)**:
  \[
  \theta_{\text{HJ}} = \sqrt{ \mu_e' \Sigma^{-1} \mu_e }.
  \]
  So in code: `theta = sqrt((mu_e @ pinv(Sigma) @ mu_e))`.

This is the slope of the Hansen–Jagannathan frontier and equals the **maximum Sharpe ratio** achievable by any portfolio of these assets. In mean–variance theory the **tangency portfolio** is the portfolio that attains exactly this maximum, and its weights are proportional to \( \Sigma^{-1} \mu_e \). So **by construction**, the tangency portfolio’s Sharpe ratio equals \( \theta_{\text{HJ}} \) when both are computed from the same \( \mu, \Sigma, r_f \).

---

## Why these results add no value

1. **Same number**: In-sample we get \( \theta_{\text{HJ}} = 0.3368 \) (30 ind) and \( 0.3995 \) (FF5), and the **tangency Sharpe equals these exactly**. That is because we use the same \( \mu, \Sigma, r_f \) for both: the HJ bound is just the tangency Sharpe written in the form \( \sqrt{\mu_e' \Sigma^{-1} \mu_e} \). There is no separate “bound” here—it’s the same quantity.

2. **Cap never binds**: The “HJ-capped” portfolio is defined as: if tangency Sharpe ≤ θ_HJ, keep tangency; otherwise take the portfolio on the GMV–TAN segment with Sharpe = θ_HJ. Since tangency Sharpe = θ_HJ, we always keep tangency. So **w_cap = w_tan** and the “HJ-capped” portfolio is **identical** to the tangency portfolio.

3. **Identical OOS**: Same weights ⇒ same OOS returns ⇒ same OOS Sharpe (0.0813 and 0.1987) and same frontier replication R² (0.1116, 0.1107). We are comparing the tangency to itself. The JK/LW tests are just testing that one portfolio’s Sharpe drops from IS to OOS.

**Conclusion**: In this setup the HJ bound does not act as an extra constraint. To get something useful you would need a setting where some **other** portfolio (e.g. from resampling, or a different estimator) has **estimated** Sharpe **above** the estimated θ_HJ, so that capping at θ_HJ actually changes the chosen portfolio; or to use a **different** estimate for the bound (e.g. different sample or method) than for the tangency so the cap can bind.

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import brentq

from fin41360.process_french import load_industry_30_monthly, load_ff5_monthly
from fin41360.mv_frontier import (
    compute_moments_from_net,
    gmv_weights,
    tangency_weights,
    portfolio_stats,
    portfolio_sharpe,
    hj_bound,
)
from fin41360.sharpe_tests import jobson_korkie_test, ledoit_wolf_test, frontier_replication_alpha

END_IS, START_OOS, END_OOS = "2002-12", "2003-01", "2025-12"
ind_is = load_industry_30_monthly(end=END_IS)
ind_oos = load_industry_30_monthly(start=START_OOS, end=END_OOS)
ff5_is, rf_is = load_ff5_monthly(end=END_IS)
ff5_oos, rf_oos = load_ff5_monthly(start=START_OOS, end=END_OOS)

ind_net_is = ind_is.values - 1.0
ind_net_oos = ind_oos.values - 1.0
rf_is_net = (rf_is - 1.0).values
rf_oos_net = (rf_oos - 1.0).values
rf_ind_is = float(np.mean(rf_is_net))
rf_oos_scalar = float(np.mean(rf_oos_net))

mu_ind_is, Sigma_ind_is = compute_moments_from_net(ind_net_is)
mu_ind_oos, Sigma_ind_oos = compute_moments_from_net(ind_net_oos)
ff5_is_arr = ff5_is.values
ff5_oos_arr = ff5_oos.values
mu_ff5_is, Sigma_ff5_is = compute_moments_from_net(ff5_is_arr)
mu_ff5_oos, Sigma_ff5_oos = compute_moments_from_net(ff5_oos_arr)

In [2]:
# HJ bound (max Sharpe) in-sample
theta_ind = hj_bound(mu_ind_is, Sigma_ind_is, rf_ind_is)
theta_ff5 = hj_bound(mu_ff5_is, Sigma_ff5_is, 0.0)

w_tan_ind = tangency_weights(mu_ind_is, Sigma_ind_is, rf_ind_is)
w_gmv_ind = gmv_weights(Sigma_ind_is)
sr_tan_ind = portfolio_sharpe(w_tan_ind, mu_ind_is, Sigma_ind_is, rf_ind_is)

w_tan_ff5 = tangency_weights(mu_ff5_is, Sigma_ff5_is, 0.0)
w_gmv_ff5 = gmv_weights(Sigma_ff5_is)
sr_tan_ff5 = portfolio_sharpe(w_tan_ff5, mu_ff5_is, Sigma_ff5_is, 0.0)

print("In-sample HJ bound (max Sharpe): 30 ind = {:.4f}, FF5 = {:.4f}".format(theta_ind, theta_ff5))
print("In-sample tangency Sharpe:       30 ind = {:.4f}, FF5 = {:.4f}".format(sr_tan_ind, sr_tan_ff5))

In-sample HJ bound (max Sharpe): 30 ind = 0.3368, FF5 = 0.3995
In-sample tangency Sharpe:       30 ind = 0.3368, FF5 = 0.3995


In [3]:
def weights_capped_sharpe(mu, Sigma, rf, w_tan, w_gmv, max_sharpe):
    """Portfolio on GMV--TAN segment with Sharpe = min(SR_tan, max_sharpe)."""
    sr_tan = portfolio_sharpe(w_tan, mu, Sigma, rf)
    if sr_tan <= max_sharpe:
        return w_tan.copy()
    def sharpe_diff(alpha):
        w = (1 - alpha) * w_gmv + alpha * w_tan
        return portfolio_sharpe(w, mu, Sigma, rf) - max_sharpe
    try:
        alpha = brentq(sharpe_diff, 1e-6, 1.0)
    except Exception:
        return w_tan.copy()
    return (1 - alpha) * w_gmv + alpha * w_tan

w_cap_ind = weights_capped_sharpe(mu_ind_is, Sigma_ind_is, rf_ind_is, w_tan_ind, w_gmv_ind, theta_ind)
w_cap_ff5 = weights_capped_sharpe(mu_ff5_is, Sigma_ff5_is, 0.0, w_tan_ff5, w_gmv_ff5, theta_ff5)
print("30 ind: HJ-capped Sharpe = {:.4f}".format(portfolio_sharpe(w_cap_ind, mu_ind_is, Sigma_ind_is, rf_ind_is)))
print("FF5:    HJ-capped Sharpe = {:.4f}".format(portfolio_sharpe(w_cap_ff5, mu_ff5_is, Sigma_ff5_is, 0.0)))

30 ind: HJ-capped Sharpe = 0.3368
FF5:    HJ-capped Sharpe = 0.3995


In [4]:
r_tan_ind_oos = ind_net_oos @ w_tan_ind
r_cap_ind_oos = ind_net_oos @ w_cap_ind
r_tan_ff5_oos = ff5_oos_arr @ w_tan_ff5
r_cap_ff5_oos = ff5_oos_arr @ w_cap_ff5

def oos_sharpe(rets, rf):
    e = rets - rf
    return np.mean(e) / np.std(e, ddof=1) if np.std(e, ddof=1) > 0 else np.nan

sr_tan_ind_oos = oos_sharpe(r_tan_ind_oos, rf_oos_net)
sr_cap_ind_oos = oos_sharpe(r_cap_ind_oos, rf_oos_net)
sr_tan_ff5_oos = np.mean(r_tan_ff5_oos) / np.std(r_tan_ff5_oos, ddof=1)
sr_cap_ff5_oos = np.mean(r_cap_ff5_oos) / np.std(r_cap_ff5_oos, ddof=1)

print("Out-of-sample Sharpe:")
print("  30 ind: TAN = {:.4f}, HJ-capped = {:.4f}".format(sr_tan_ind_oos, sr_cap_ind_oos))
print("  FF5:    TAN = {:.4f}, HJ-capped = {:.4f}".format(sr_tan_ff5_oos, sr_cap_ff5_oos))

Out-of-sample Sharpe:
  30 ind: TAN = 0.0813, HJ-capped = 0.0813
  FF5:    TAN = 0.1987, HJ-capped = 0.1987


In [5]:
w_tan_ind_oos = tangency_weights(mu_ind_oos, Sigma_ind_oos, rf_oos_scalar)
w_gmv_ind_oos = gmv_weights(Sigma_ind_oos)
w_tan_ff5_oos = tangency_weights(mu_ff5_oos, Sigma_ff5_oos, 0.0)
w_gmv_ff5_oos = gmv_weights(Sigma_ff5_oos)

rep_tan_ind = frontier_replication_alpha(w_tan_ind, w_tan_ind_oos, w_gmv_ind_oos)
rep_cap_ind = frontier_replication_alpha(w_cap_ind, w_tan_ind_oos, w_gmv_ind_oos)
rep_tan_ff5 = frontier_replication_alpha(w_tan_ff5, w_tan_ff5_oos, w_gmv_ff5_oos)
rep_cap_ff5 = frontier_replication_alpha(w_cap_ff5, w_tan_ff5_oos, w_gmv_ff5_oos)

print("Frontier replication R² (1 = on OOS frontier):")
print("  30 ind: TAN R² = {:.4f}, HJ-capped R² = {:.4f}".format(rep_tan_ind.r_squared, rep_cap_ind.r_squared))
print("  FF5:    TAN R² = {:.4f}, HJ-capped R² = {:.4f}".format(rep_tan_ff5.r_squared, rep_cap_ff5.r_squared))

Frontier replication R² (1 = on OOS frontier):
  30 ind: TAN R² = 0.1116, HJ-capped R² = 0.1116
  FF5:    TAN R² = 0.1107, HJ-capped R² = 0.1107


In [6]:
# Optional: Jobson–Korkie and Ledoit–Wolf for HJ-capped (IS vs OOS Sharpe)
r_cap_ind_is = ind_net_is @ w_cap_ind
r_cap_ff5_is = ff5_is_arr @ w_cap_ff5
jk_cap_ind = jobson_korkie_test(r_cap_ind_is, r_cap_ind_oos, rf_ind_is, rf_oos_scalar)
lw_cap_ind = ledoit_wolf_test(r_cap_ind_is, r_cap_ind_oos, rf_ind_is, rf_oos_scalar, n_boot=2000)
jk_cap_ff5 = jobson_korkie_test(r_cap_ff5_is, r_cap_ff5_oos, 0.0, 0.0)
lw_cap_ff5 = ledoit_wolf_test(r_cap_ff5_is, r_cap_ff5_oos, 0.0, 0.0, n_boot=2000)
print("HJ-capped: JK p-value 30 ind = {:.4f}, FF5 = {:.4f}".format(jk_cap_ind.pvalue_two_sided, jk_cap_ff5.pvalue_two_sided))
print("HJ-capped: LW p-value 30 ind = {:.4f}, FF5 = {:.4f}".format(lw_cap_ind.pvalue_two_sided, lw_cap_ff5.pvalue_two_sided))

HJ-capped: JK p-value 30 ind = 0.0031, FF5 = 0.0213
HJ-capped: LW p-value 30 ind = 0.0010, FF5 = 0.0400
